In [2]:
#O coeficiente de correlação linear tem um valor entre -1 e 1.
#Um valor 0 indica que não há correlação linear entre os dois mapas.
#Valores próximos a zero indicam uma fraca correlação entre os dois conjuntos.
#Um valor 1 indica uma correlação perfeita. 

import glob
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import numpy

count = 0
q_all = []
p_all = []
out_all = []
media = 0
out = 0

def normalize(x, method='standard', axis=None):
	x = np.array(x, copy=False)
	if axis is not None:
		y = np.rollaxis(x, axis).reshape([x.shape[axis], -1])
		shape = np.ones(len(x.shape))
		shape[axis] = x.shape[axis]
		if method == 'standard':
			res = (x - np.mean(y, axis=1).reshape(shape)) / np.std(y, axis=1).reshape(shape)
		elif method == 'range':
			res = (x - np.min(y, axis=1).reshape(shape)) / (np.max(y, axis=1) - np.min(y, axis=1)).reshape(shape)
		elif method == 'sum':
			res = x / np.float_(np.sum(y, axis=1).reshape(shape))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')
	else:
		if method == 'standard':
			res = (x - np.mean(x)) / np.std(x)
		elif method == 'range':
			res = (x - np.min(x)) / (np.max(x) - np.min(x))
		elif method == 'sum':
			res = x / float(np.sum(x))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')

def CC(saliency_map1, saliency_map2):   
	#'''
	#Coeficiente de correlação de Pearson entre dois mapas de saliência diferentes
	#(CC=0 para mapas não correlacionados, CC=1 para correlação linear perfeita).
	#Parâmetros
	#----------
	#saliency_map1 : matriz de valor real
	#	Se os dois mapas forem diferentes em forma, saliency_map1 será redimensionado para corresponder a saliency_map2.
	#saliency_map2 : matriz de valor real
	#Devoluções
	#-------
	#CC : float, entre [-1,1]
	#'''
     
	map1 = np.array(saliency_map1, copy=False)
	map2 = np.array(saliency_map2, copy=False)
	if map1.shape != map2.shape:
		map1 = resize(map1, map2.shape, order=3, mode='constant') # bi-cubic/nearest is what Matlab imresize() does by default
	# Normalize the two maps to have zero mean and unit std
	#map1 = normalize(map1, method='standard')
	#map2 = normalize(map2, method='standard')
	# Compute correlation coefficient
	return np.corrcoef(map1.ravel(), map2.ravel())[0,1]

if __name__ == '__main__':
    folder = r'...\saliency_all' #Diretório que contenha os mapas de saliência e os grounds truth

    gt_saliency_folder = folder + '\salmap' #mapas de saliencia do modelo
    gt_fixation_folder = folder + '\\fixmap' #mapas de saliencia - ground truth
    #print(gt_saliency_folder)
     
            
    for filename_sal in glob.glob(os.path.join(gt_saliency_folder,'*.png')):
        #print(filename_sal)
        p= plt.imread(filename_sal)
        p_all.append(p)
    for filename_fix in glob.glob(os.path.join(gt_fixation_folder,'*.jpeg')):
        #print(filename_fix)
        q= plt.imread(filename_fix)
        q_all.append(q)    
                   
    for i in range(0,len(p_all)):
        #print(p_all[3].shape)
        out = CC(p_all[i],p_all[i])
        out_all.append(out)
    soma = sum(out_all)
    media = soma/len(p_all)
    print (media) 

1.0
